# 10-4 ImageFolder (2)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)

In [4]:
data_loader = DataLoader(dataset = train_data,
                         batch_size=8,
                         shuffle=True,
                         num_workers=2)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # input image (3, 64, 64)
        # Conv2d (6, 60, 60)
        # MaxPool2d (6, 30, 30)
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, 5),  # in_channel, out_channel, kernel (no padding)
            nn.ReLU(),           # ReLU activation function
            nn.MaxPool2d(2),     # kernel size (stride default value = kernel size)
        )
        # input (6, 30, 30)
        # Conv2d (16, 26, 26)
        # MaxPool2d (16, 13, 13)
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        # input (16, 13, 13)
        # Linear (100)
        # Linear (2)
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*13, 100),
            nn.ReLU(),
            nn.Linear(100, 2)
        )
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out

In [6]:
# testing
net = CNN().to(device)
test_input = (torch.Tensor(3, 3, 64, 64)).to(device)
test_out = net(test_input)

In [7]:
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [8]:
total_batch = len(data_loader)

epochs = 100
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        out = net(imgs)
        loss = loss_func(out, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
    
    if epoch % 10 == 0:
        print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.6951799988746643
[Epoch:11] cost = 0.563401997089386
[Epoch:21] cost = 0.4471661150455475
[Epoch:31] cost = 0.3532431423664093
[Epoch:41] cost = 0.3071574866771698
[Epoch:51] cost = 0.2397182583808899
[Epoch:61] cost = 0.21134188771247864
[Epoch:71] cost = 0.18309545516967773
[Epoch:81] cost = 0.1704423874616623
[Epoch:91] cost = 0.13286486268043518
Learning Finished!


In [9]:
# save model
torch.save(net.state_dict(), "./model/model.pth")

In [10]:
new_net = CNN().to(device)

In [11]:
# load model
new_net.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [12]:
# check saved model and loaded model are same
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0]

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0844,  0.0100, -0.0109, -0.0154,  0.0981], device='cuda:0',
       grad_fn=<SelectBackward>)
tensor([-0.0844,  0.0100, -0.0109, -0.0154,  0.0981], device='cuda:0',
       grad_fn=<SelectBackward>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

In [13]:
tran=torchvision.transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='custom_data/test_data', transform=tran)

In [14]:
test_set = DataLoader(dataset=test_data, batch_size=len(test_data))

In [15]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.8500000238418579
